In [ ]:
# запускаю поставку данных в режиме реального времени
!curl -X POST https://order-gen-service.sprint9.tgcloudenv.ru/project/register_kafka \
-H 'Content-Type: application/json; charset=utf-8' \
--data-binary @- << EOF
{
    "student": "producer_consumer",
    "kafka_connect":{
        "host": "rc1a-hankn22hfdvk7pa3.mdb.yandexcloud.net",
        "port": 9091,
        "topic": "transaction-service-input",
        "producer_name": "producer_consumer",
        "producer_password": "Jees3Xei"
    }
}
EOF

In [ ]:
# режим консьюмера - получаем сообщения в топик transaction-service-input
!sudo docker run \
    -it \
    --name "kcat" \
    --network=host \
    --rm \
    -v "/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt:/data/CA.pem" \
    edenhill/kcat:1.7.1 -b rc1a-hankn22hfdvk7pa3.mdb.yandexcloud.net:9091 \
    -X security.protocol=SASL_SSL \
    -X sasl.mechanisms=SCRAM-SHA-512 \
    -X sasl.username=producer_consumer \
    -X sasl.password="Jees3Xei" \
    -X ssl.ca.location=/data/CA.pem \
    -t transaction-service-input \
    -C \
    -o beginning

In [ ]:
# изучаем данные из источника кафка
{
  "object_id": "256409aa-b5cd-4425-ae1a-bb9c53913d07",
  "object_type": "TRANSACTION",
  "sent_dttm": "2022-10-01T08:34:19",
  "payload": {
    "operation_id": "256409aa-b5cd-4425-ae1a-bb9c53913d07",
    "account_number_from": 914810,
    "account_number_to": 5735373,
    "currency_code": 430,
    "country": "russia",
    "status": "chargeback",
    "transaction_type": "c2a_incoming",
    "amount": 1500000,
    "transaction_dt": "2022-10-01 08:34:19"
  }
}

### Структура данных TRANSACTION:

- operation_id — id транзакции;
- account_number_from — внутренний бухгалтерский номер счёта транзакции ОТ КОГО;
- account_number_to — внутренний бухгалтерский номер счёта транзакции К КОМУ;
- currency_code — трёхзначный код валюты страны, из которой идёт транзакция;
- country — страна-источник транзакции;
- status — статус проведения транзакции: queued («транзакция в очереди на обработку сервисом»), in_progress («транзакция в обработке»), blocked («транзакция заблокирована сервисом»), done («транзакция выполнена успешно»), chargeback («пользователь осуществил возврат по транзакции»).
- transaction_type — тип транзакции во внутреннем учёте: authorisation («авторизационная транзакция, подтверждающая наличие счёта пользователя»), sbp_incoming («входящий перевод по системе быстрых платежей»), sbp_outgoing («исходящий перевод по системе быстрых платежей»), transfer_incoming («входящий перевод по счёту»), transfer_outgoing («исходящий перевод по счёту»), c2b_partner_incoming («перевод от юридического лица»), c2b_partner_outgoing («перевод юридическому лицу»).
- amount — целочисленная сумма транзакции в минимальной единице валюты страны (копейка, цент, куруш);
- transaction_dt — дата и время исполнения транзакции до миллисекунд.

### Структура данных сurrencies:

- date_update — дата обновления курса валют;
- currency_code — трёхзначный код валюты транзакции;
- currency_code_with — отношение другой валюты к валюте трёхзначного кода;
- currency_code_div — значение отношения единицы одной валюты к единице валюты транзакции.

In [15]:
# проверка подключения к вертике
import vertica_python
from getpass import getpass

conn_info = {'host': '51.250.75.20',
             'port': 5433,
             'user': 'YAMARSELR2015YANDEXRU',
             'password': 'z0shuiL3wY6yb8y',
             'database': 'dwh',
             'autocommit': True
}

with vertica_python.connect(**conn_info) as conn:
    print('yo')

ConnectionError: Severity: FATAL, Message: Invalid username or password, Sqlstate: 28000, Routine: auth_failed, File: /data/jenkins/workspace/RE-ReleaseBuilds/RE-Jackhammer/server/vertica/Basics/ClientAuthentication.cpp, Line: 1050, Error Code: 3781

In [2]:
# Локально запустить докер
sudo docker compose up -d --build

# Подготовьте kubeconfig
export KUBECONFIG=/home/user/.kube/config

# Собрать образ -> запушить в реестр
sudo docker build . -t cr.yandex/crp5rbbb4epngincu3l9/stg_service:v2023-04-24-r1
sudo docker push cr.yandex/crp5rbbb4epngincu3l9/stg_service:v2023-04-24-r1
# (дать права на пушинг)

# Пропишите тег в values.yaml
tag: "v2023-04-24-r1" 
    
# Выполните релиз сервиса.
helm upgrade --install stg-service app -n c08-marsel-kamilov

# Проверьте с помощью kubectl, что сервис запущен и работает
kubectl get deployment
kubectl get pods
kubectl describe pod
kubectl describe deployment stg-service

# если нужно удалить деплоймент
helm uninstall stg-service

# просмотра состояния релизов
helm list

SyntaxError: invalid syntax (2766649448.py, line 1)

In [1]:
# Postgresql
# тест подключения
curl -X POST https://postgres-check-service.sprint9.tgcloudenv.ru/init_schemas \
-H 'Content-Type: application/json; charset=utf-8' \
--data-binary @- << EOF
{
  "student": "student",
  "pg_settings": {
    "host": "rc1b-c93bouyn77cmtsrj.mdb.yandexcloud.net",
    "port": 6432,
    "dbname": "sprint9dwh",
    "username": "db_user",
    "password": "Jees3Xei"
  }
}
EOF

In [ ]:
# Helm
# установка
curl -fsSL -o get_helm.sh https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3
chmod 700 get_helm.sh
./get_helm.sh
# запуск релиза
helm upgrade --install sprint9-test /home/user/Downloads/Cources/YandexPracticum/DE/sprint9-demo-service/de_cloudtech_demo_service/app
# установка приложения в кластер kubernetes. Теперь kubernetes может управлять эти приложением
kubectl exec manifest-5d9779bc46-55nlv -- /src/scripts/dist/helm_test

In [ ]:
# Kubernetes, kubestl, manifests
# запуск деплоймента (объектов)
kubectl create -f deployment.yml
# просмотра состояния объектов
kubectl get deployment
# просмотра состояния подов
kubectl get pods
# проверка, что пода запущена
kubectl exec manifest-5d9779bc46-55nlv -- /src/scripts/dist/manifest_test

In [1]:
# Репозиторий для образов контейнеров
yc --version
yc init
yc config list
yc vpc network list
yc container registry configure-docker
sudo docker pull gcr.io/google_containers/pause
sudo docker tag gcr.io/google_containers/pause cr.yandex/crp5rbbb4epngincu3l9/test:2023-04-20-v1
sudo docker push cr.yandex/crp5rbbb4epngincu3l9/test:2023-04-20-v1

NameError: name 'yc' is not defined

In [ ]:
# отправка сообщения в кафку для теста
!curl -X POST https://order-gen-service.sprint9.tgcloudenv.ru/test_kafka \
-H 'Content-Type: application/json; charset=utf-8' \
--data-binary @- << EOF
{
    "student": "producer_consumer",
    "kafka_connect":{
        "host": "rc1a-hankn22hfdvk7pa3.mdb.yandexcloud.net",
        "port": 9091,
        "topic": "transaction-service-input",
        "producer_name": "producer_consumer",
        "producer_password": "Jees3Xei"
    }
}
EOF

In [ ]:
# Подключитесь через kcat к Kafka. 
# Чтобы убедиться, что параметры подключения прописаны корректно, 
# получите также список всех топиков, созданных в вашей инсталляции Kafka
!sudo docker run \
    -it \
    --network=host \
    -v "/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt:/data/CA.pem" \
    edenhill/kcat:1.7.1 -b rc1a-hankn22hfdvk7pa3.mdb.yandexcloud.net:9091 \
    -X security.protocol=SASL_SSL \
    -X sasl.mechanisms=SCRAM-SHA-512 \
    -X sasl.username=producer_consumer \
    -X sasl.password="Jees3Xei" \
    -X ssl.ca.location=/data/CA.pem \
    -L

In [ ]:
# kafkacat -C \
#          -b rc1a-hankn22hfdvk7pa3.mdb.yandexcloud.net:9091 \
#          -t "1" \
#          -X security.protocol=SASL_SSL \
#          -X sasl.mechanisms=SCRAM-SHA-512 \
#          -X sasl.username="producer_consumer" \
#          -X sasl.password="Jees3Xei" \
#          -X ssl.ca.location="/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt" -Z -K:

In [37]:
# Kafka settings
host_kafka="rc1a-hankn22hfdvk7pa3.mdb.yandexcloud.net",
port_kafka=9091,
user_kafka="producer_consumer",
password_kafka="Jees3Xei",
ssl_kafka=True,
ssl_ca_certs_kafka="/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt"

In [106]:
df_new = pd.merge(df, df2, on=['date_list_transactions'], how='right', indicator=True)

KeyError: False

In [104]:
df2

,date_list_transactions
0,2022-12-16 17:33:26.466
1,2022-12-16 17:33:26.196
2,2022-12-16 17:33:34.783


In [103]:
data = {"date_list_transactions": date_list_transactions[:3]}
df2 = pd.DataFrame(data)

In [102]:
df

,date_list_transactions
0,2022-12-16 17:33:26.466
1,2022-12-16 17:33:26.196


In [101]:
data = {"date_list_transactions": date_list_transactions[:2]}
data
df = pd.DataFrame(data)

In [100]:
import numpy as np
import datetime 

date_list_transactions = [datetime.datetime(2022, 12, 16, 17, 33, 26, 466000), 
         datetime.datetime(2022, 12, 16, 17, 33, 26, 196000), 
         datetime.datetime(2022, 12, 16, 17, 33, 34, 783000), 
         datetime.datetime(2022, 12, 16, 17, 33, 34, 756000), 
         datetime.datetime(2022, 12, 16, 17, 33, 34, 772000), 
         datetime.datetime(2022, 12, 16, 17, 33, 34, 789000), 
         datetime.datetime(2022, 12, 16, 17, 33, 26, 227000), 
         datetime.datetime(2022, 12, 16, 17, 33, 26, 183000), 
         datetime.datetime(2022, 12, 16, 17, 33, 26, 307000), 
         datetime.datetime(2022, 12, 16, 17, 33, 26, 169000)]

csv_list_transactions = ['transactions_batch_1.csv', 
       'transactions_batch_10.csv', 
       'transactions_batch_2.csv', 
       'transactions_batch_3.csv', 
       'transactions_batch_4.csv', 
       'transactions_batch_5.csv', 
       'transactions_batch_6.csv', 
       'transactions_batch_7.csv', 
       'transactions_batch_8.csv', 
       'transactions_batch_9.csv']

ids_sorted = [ids[j] for _,j in enumerate(np.argsort(dates))]
ids_sorted

['transactions_batch_9.csv',
 'transactions_batch_7.csv',
 'transactions_batch_10.csv',
 'transactions_batch_6.csv',
 'transactions_batch_8.csv',
 'transactions_batch_1.csv',
 'transactions_batch_3.csv',
 'transactions_batch_4.csv',
 'transactions_batch_2.csv',
 'transactions_batch_5.csv']

In [78]:
batch_file_names

['transactions_batch_1.csv']

In [79]:
batch_file_change_date

['0']

In [77]:
batch_file_names = df_comp[df_comp['_merge'] == 'right_only']['batch_file_name'].to_list()
batch_file_change_date = df_comp[df_comp['_merge'] == 'right_only']['batch_file_change_date'].to_list()

In [74]:
df_comp = pd.merge(df[['batch_file_name','batch_file_change_date']], df_inc[['batch_file_name','batch_file_change_date']], on=['batch_file_name','batch_file_change_date'], how='right', indicator=True)
df_comp

,batch_file_name,batch_file_change_date,_merge
0,transactions_batch_1.csv,0,right_only
1,transactions_batch_10.csv,2022-12-16 17:33:26.196000,both
2,transactions_batch_2.csv,2022-12-16 17:33:34.783000,both
3,transactions_batch_3.csv,2022-12-16 17:33:34.756000,both
4,transactions_batch_4.csv,2022-12-16 17:33:34.772000,both
5,transactions_batch_5.csv,2022-12-16 17:33:34.789000,both
6,transactions_batch_6.csv,2022-12-16 17:33:26.227000,both
7,transactions_batch_7.csv,2022-12-16 17:33:26.183000,both
8,transactions_batch_8.csv,2022-12-16 17:33:26.307000,both
9,transactions_batch_9.csv,2022-12-16 17:33:26.169000,both


In [59]:
df_inc

,id,date_reading,batch_file_name,batch_file_change_date
0,0,2023-05-06 18:03:04.153815,transactions_batch_1.csv,0
1,1,2023-05-06 18:03:04.153832,transactions_batch_10.csv,2022-12-16 17:33:26.196000
2,2,2023-05-06 18:03:04.153838,transactions_batch_2.csv,2022-12-16 17:33:34.783000
3,3,2023-05-06 18:03:04.153842,transactions_batch_3.csv,2022-12-16 17:33:34.756000
4,4,2023-05-06 18:03:04.153846,transactions_batch_4.csv,2022-12-16 17:33:34.772000
5,5,2023-05-06 18:03:04.153850,transactions_batch_5.csv,2022-12-16 17:33:34.789000
6,6,2023-05-06 18:03:04.153855,transactions_batch_6.csv,2022-12-16 17:33:26.227000
7,7,2023-05-06 18:03:04.153859,transactions_batch_7.csv,2022-12-16 17:33:26.183000
8,8,2023-05-06 18:03:04.153863,transactions_batch_8.csv,2022-12-16 17:33:26.307000
9,9,2023-05-06 18:03:04.153867,transactions_batch_9.csv,2022-12-16 17:33:26.169000


In [58]:
df_inc['batch_file_change_date'].iloc[0] = '0'

/tmp/ipykernel_34693/2265244066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inc['batch_file_change_date'].iloc[0] = '0'


In [46]:
df_inc = df.copy()
df_inc

,id,date_reading,batch_file_name,batch_file_change_date
0,0,2023-05-06 18:03:04.153815,transactions_batch_1.csv,2022-12-16 17:33:26.466000
1,1,2023-05-06 18:03:04.153832,transactions_batch_10.csv,2022-12-16 17:33:26.196000
2,2,2023-05-06 18:03:04.153838,transactions_batch_2.csv,2022-12-16 17:33:34.783000
3,3,2023-05-06 18:03:04.153842,transactions_batch_3.csv,2022-12-16 17:33:34.756000
4,4,2023-05-06 18:03:04.153846,transactions_batch_4.csv,2022-12-16 17:33:34.772000
5,5,2023-05-06 18:03:04.153850,transactions_batch_5.csv,2022-12-16 17:33:34.789000
6,6,2023-05-06 18:03:04.153855,transactions_batch_6.csv,2022-12-16 17:33:26.227000
7,7,2023-05-06 18:03:04.153859,transactions_batch_7.csv,2022-12-16 17:33:26.183000
8,8,2023-05-06 18:03:04.153863,transactions_batch_8.csv,2022-12-16 17:33:26.307000
9,9,2023-05-06 18:03:04.153867,transactions_batch_9.csv,2022-12-16 17:33:26.169000


In [99]:
df = pd.DataFrame(columns=['id','date_reading','batch_file_name','batch_file_change_date'])
df.empty

True

In [89]:
df1 = df[df["batch_file_name"]=="transactions_batch_1.csv"]
df1

,id,date_reading,batch_file_name,batch_file_change_date
0,0,2023-05-06 18:03:04.153815,transactions_batch_1.csv,2022-12-16 17:33:26.466000


In [93]:
df

,id,date_reading,batch_file_name,batch_file_change_date
0,0,2023-05-06 18:03:04.153815,transactions_batch_1.csv,2022-12-16 17:33:26.466000
1,1,2023-05-06 18:03:04.153832,transactions_batch_10.csv,2022-12-16 17:33:26.196000
2,2,2023-05-06 18:03:04.153838,transactions_batch_2.csv,2022-12-16 17:33:34.783000
3,3,2023-05-06 18:03:04.153842,transactions_batch_3.csv,2022-12-16 17:33:34.756000
4,4,2023-05-06 18:03:04.153846,transactions_batch_4.csv,2022-12-16 17:33:34.772000
5,5,2023-05-06 18:03:04.153850,transactions_batch_5.csv,2022-12-16 17:33:34.789000
6,6,2023-05-06 18:03:04.153855,transactions_batch_6.csv,2022-12-16 17:33:26.227000
7,7,2023-05-06 18:03:04.153859,transactions_batch_7.csv,2022-12-16 17:33:26.183000
8,8,2023-05-06 18:03:04.153863,transactions_batch_8.csv,2022-12-16 17:33:26.307000
9,9,2023-05-06 18:03:04.153867,transactions_batch_9.csv,2022-12-16 17:33:26.169000


In [41]:
from datetime import datetime
import pandas as pd
data = {
    "id": list(range(next_id, next_id + len(csv_list_transactions))),
    "date_reading": [
        str(datetime.today()) for _ in range(len(csv_list_transactions))
    ],
    "batch_file_name": csv_list_transactions,
    "batch_file_change_date": [str(i) for i in date_list_transactions],
}

df = pd.DataFrame(data)

In [35]:
next_id = 0
list(range(next_id, next_id + 10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]